<img src="Quantum_Brilliance_dark_blue_Logo_RGB.png" alt="Copyright (c) 2022 Quantum Brilliance Pty Ltd" width="240">


## Example of a custom ansatz for VQE in Qristal

In [1]:
import qbos_op
from scipy.optimize import minimize, show_options, Bounds

### Function for scipy.optimize to call

In [2]:
def qbvqe(theta,m_qv):
    for el in range(len(theta)):
        m_qv.theta[0][0][el] = theta[el]
    m_qv.run()
    return m_qv.out_energy[0][0][0]

### Custom ansatz, in XASM format

**Important:**

Do **not** change the first two lines:
```
.compiler xasm
.circuit qbos_ansatz
```

You can use the parameter (declared here as `theta`) as a 0-indexed array, ie `theta[0], theta[1], theta[2]`, ..., etc.

In [3]:
qbos_ansatz = '''
.compiler xasm
.circuit qbos_ansatz
.parameters theta
.qbit q
  X(q[0]);
  X(q[1]);
  U(q[0], theta[0], theta[1], theta[2]);
  U(q[1], theta[3], theta[4], theta[5]);
  U(q[2], theta[6], theta[7], theta[8]);
  U(q[3], theta[9], theta[10], theta[11]);
  CNOT(q[0], q[1]);
  CNOT(q[1], q[2]);
  CNOT(q[2], q[3]);
  U(q[0], theta[12], theta[13], theta[14]);
  U(q[1], theta[15], theta[16], theta[17]);
  U(q[2], theta[18], theta[19], theta[20]);
  U(q[3], theta[21], theta[22], theta[23]);
  CNOT(q[0], q[1]);
  CNOT(q[1], q[2]);
  CNOT(q[2], q[3]);
'''

### VQE for classical 4-qubit Hamiltonian, using built-in optimizer (Nelder-Mead)

In [4]:
qv=qbos_op.vqe()
qv.sn = 0   # No shots - deterministic VQE

# Example from user guide: 3.1.5 Example 3 - 4 qubit Hamiltonian, custom ansatz
qv.qn = 4   
qv.ham = "0.04207897647782276 + 0.17771287465139946 Z0 + 0.1777128746513994 Z1 + -0.24274280513140462 Z2 - 0.17059738328801052 Z0Z1 + 0.6622334 Z0Z1Z2Z3"
qv.ansatz = qbos_ansatz
qv.ansatz_depth = 2
qv.maxeval = 400
qv.theta = qbos_op.ND()
optimum_energy = qbvqe([0.11]*qv.ansatz_depth[0][0]*qv.qn[0][0]*3,qv)
print('Min. energy: ' + str(optimum_energy))


Min. energy: -1.3882948459026188


### VQE for classical 4-qubit Hamiltonian, using SLSQP in scipy.optimize

We first need to apply value constraints on the parameters before using SLSQP:

In [5]:
qvbound = Bounds(-3.14159,3.14159,True)

Now proceed to run ```minimize()```. 

**Important**: ensure that the number of parameters provided in the initial list matches the number required by the ansatz used.  For the default ansatz, this equals ```3*qn*ansatz_depth```:

In [6]:
qv.maxeval = 1
res = minimize(qbvqe, [0.11]*qv.ansatz_depth[0][0]*qv.qn[0][0]*3, args=(qv,), method='SLSQP', bounds=qvbound , tol=1e-6)

Inspect the optimum results returned by ```minimize()```

In [7]:
res

     fun: -1.3889202865893062
     jac: array([ 3.61949205e-05,  5.96046448e-08,  5.96046448e-08, -3.73780727e-04,
        5.96046448e-08,  5.96046448e-08,  6.31958246e-05, -4.57108021e-04,
        0.00000000e+00, -3.57940793e-04, -1.07809901e-04,  0.00000000e+00,
        1.08227134e-04, -1.49011612e-08, -1.49011612e-08,  3.95178795e-04,
        0.00000000e+00,  0.00000000e+00,  6.37620687e-05,  0.00000000e+00,
       -4.57063317e-04, -2.56091356e-04, -1.49011612e-08, -3.19927931e-05])
 message: 'Optimization terminated successfully'
    nfev: 326
     nit: 13
    njev: 13
  status: 0
 success: True
       x: array([-2.48578132e-06,  1.09853522e-01,  1.10000146e-01, -5.24819133e-04,
        1.09635322e-01,  1.10000064e-01,  1.90996417e+00, -9.69857843e-03,
        1.10000000e-01,  1.57081247e+00, -1.41775019e-04,  1.10000000e-01,
        6.17790346e-05,  1.09999815e-01,  1.09853184e-01, -5.25538051e-04,
        1.10000000e-01,  1.10267329e-01,  1.23160517e+00,  1.10000001e-01,
        

### Getting the eigenstate at the optimum theta

> **Note:** This is only valid for **classical/diagonal Hamiltonians** - please disregard otherwise

In [8]:
import qbos
import xacc
import re
mstaq = xacc.getCompiler('staq')
tqb = qbos.core()
tqb.qb12()

**Compile** the ansatz, **evaluate** it at the optimum, and **measure/observe** it, finally ending with modifications to produce a QPU kernel:

In [9]:
xacc.qasm(qbos_ansatz)
anz = xacc.getCompiled('qbos_ansatz')
anz_eval = anz.eval(res.x)
observe_all = xacc.getObservable('pauli', '1.0 Z3 Z2 Z1 Z0')

In [10]:
anz_eval_measured = observe_all.observe(anz_eval)[0]
q2oqm = re.sub(r"qreg q\[\d+\];", "", mstaq.translate(anz_eval_measured))
qbstr = '__qpu__ void QBCIRCUIT(qreg q) {\n' + q2oqm + '}'

**Inspect the QPU kernel** in OpenQASM format:

In [11]:
print(qbstr)

__qpu__ void QBCIRCUIT(qreg q) {
OPENQASM 2.0;
include "qelib1.inc";

creg q_c[4];
x q[0];
x q[1];
u3(-0.0000024857813234,0.1098535220489579,0.1100001464862537) q[0];
u3(-0.0005248191326506,0.1096353216240410,0.1100000644909175) q[1];
u3(1.9099641678099422,-0.0096985784290641,0.1100000000000000) q[2];
u3(1.5708124728497059,-0.0001417750189185,0.1100000000000000) q[3];
CX q[0], q[1];
CX q[1], q[2];
CX q[2], q[3];
u3(0.0000617790346293,0.1099998151274363,0.1098531835302538) q[0];
u3(-0.0005255380510641,0.1100000000000000,0.1102673290555136) q[1];
u3(1.2316051685918965,0.1100000011835104,0.0090951083846284) q[2];
u3(1.5707993459993970,0.1099998410247941,-0.0002204030212662) q[3];
CX q[0], q[1];
CX q[1], q[2];
CX q[2], q[3];
measure q[3] -> q_c[3];
measure q[2] -> q_c[2];
measure q[1] -> q_c[1];
measure q[0] -> q_c[0];
}


**Run the QPU kernel:**

In [ ]:
tqb.qn = 4
tqb.instring = qbstr
tqb.acc = "qpp"
tqb.run()

: 

**Print the eigenstate:**

In [13]:
tqb.out_raw[0]

String[{
    "1011": 1024
}]